In [1]:
import os
data_folder = "../input/uw-madison-gi-tract-image-segmentation/" if os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "") else "./data/"

# List all imports below
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scv_utility import *
import torch

np.random.seed(0)
torch.manual_seed(0)
pd.set_option("display.width", 120)

AttributeError: module 'torch' has no attribute '_assert'

In [2]:
# Load small train and test datasets with only stomach labels
labels = pd.read_csv(data_folder + "train.csv", converters={"id": str, "class": str, "segmentation": str})
print(f"Classes in train set: {labels['class'].unique()}")

train_cases = ["case2_", "case7_", "case15_", "case20_", "case22_", "case24_", "case29_", "case30_", "case32_", "case123_"]
test_cases = ["case156_", "case154_", "case149_"]
train_labels = labels[labels["id"].str.contains("|".join(train_cases))]
test_labels = labels[labels["id"].str.contains("|".join(test_cases))]

Classes in train set: ['large_bowel' 'small_bowel' 'stomach']


In [3]:
# Analyze train and test dataset to assure they have the same resolution
for sample_id in np.concatenate((train_labels["id"].unique(), test_labels["id"].unique())):
    try:
        sample_image, sample_image_res, sample_pixel_size = get_image_data_from_id(sample_id, data_folder)
        #print(f"Image shape: {sample_image.shape}, reported resolution: {sample_image_res}, reported pixel size: {sample_pixel_size}")
        assert sample_image_res == (266, 266) and sample_pixel_size == 1.50, "Incorrect resolution or pixel size"
    except Exception as e:
        print(f"Exception {e} while reading image {sample_id}")
print("Dataset analysis successfull")

Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0001
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0002
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0003
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0004
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0005
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0006
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0007
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0008
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0009
Exception name 'get_image_data_from_id' is not defined while reading image case123_day20_slice_0010


Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0087
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0088
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0089
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0090
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0091
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0092
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0093
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0094
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0095
Exception name 'get_image_data_from_id' is not defined while reading image case24_day0_slice_0096
Exception name 'get_

Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0003
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0004
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0005
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0006
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0007
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0008
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0009
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0010
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0011
Exception name 'get_image_data_from_id' is not defined while reading image case2_day5_slice_0012
Exception name 'get_image_data

Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0038
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0039
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0040
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0041
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0042
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0043
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0044
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0045
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0046
Exception name 'get_image_data_from_id' is not defined while reading image case149_day15_slice_0047


## Classification network

In [ ]:
# Training parameters
batch_size = 32
learning_rate = 0.01
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 3

# Try using gpu instead of cpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

train_labels = train_labels[(train_labels["class"] == "stomach")]
test_labels = test_labels[(test_labels["class"] == "stomach")]
train_data = MRIClassificationDataset(data_folder, train_labels)
test_data = MRIClassificationDataset(data_folder, test_labels)
print(f"Number of train images: {len(train_data)}, test images: {len(test_data)}")

# Initialize network
from torchvision.models import resnet50
net = torch.nn.Sequential(resnet50(pretrained=False), torch.nn.Linear(1000, 1))
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Training loop
train(net, train_data, test_data, criterion, optimizer, batch_size, epochs, "classifier")

# TODO calculate accuracy, precision, recall, etc on test set

## Segmentation network

In [ ]:
# Training parameters
batch_size = 32
learning_rate = 0.01
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 3

# Try using gpu instead of cpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

train_labels = train_labels[(train_labels["class"] == "stomach") & (train_labels["segmentation"] != "")]
test_labels = test_labels[(test_labels["class"] == "stomach") & (test_labels["segmentation"] != "")]
train_data = MRISegmentationDataset(data_folder, train_labels)
test_data = MRISegmentationDataset(data_folder, test_labels)
print(f"Number of train images: {len(train_data)}, test images: {len(test_data)}")

# Initialize network
from torchvision.models.segmentation import fcn_resnet50
net = fcn_resnet50(pretrained=False, num_classes=1)
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Training loop
train(net, train_data, test_data, criterion, optimizer, batch_size, epochs, "segmentation", lambda out : out["out"])

# TODO calculate segmentation metrics on test set

## Segmentation + Augmentation

In [ ]:
# Load small train and test datasets with only stomach labels
labels = pd.read_csv(data_folder + "train.csv", converters={"id": str, "class": str, "segmentation": str})
print(f"Classes in train set: {labels['class'].unique()}")

train_cases = ["case2_", "case6_", "case7_", "case9_", "case11_", "case15_", "case16_", "case18_","case19_","case20_", "case22_", "case24_", "case29_", "case30_", "case32_", "case33_", "case34_", "case35_","case123_"]
test_cases = ["case156_", "case154_", "case149_"]
train_labels = labels[labels["id"].str.contains("|".join(train_cases))]
test_labels = labels[labels["id"].str.contains("|".join(test_cases))]

In [ ]:
from torchvision import transforms
# Training parameters
batch_size = 32
learning_rate = 0.01
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 3

# Try using gpu instead of cpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

train_labels = train_labels[(train_labels["class"] == "stomach") & (train_labels["segmentation"] != "")]
test_labels = test_labels[(test_labels["class"] == "stomach") & (test_labels["segmentation"] != "")]
train_data = MRISegmentationDataset(data_folder, train_labels, transform = transforms.Compose([
                                               Rescale((266,266)), 
                                               RandomCrop(),
                                               LabelSmoothing(p=0.4)]))
test_data = MRISegmentationDataset(data_folder, test_labels)
print(f"Number of train images: {len(train_data)}, test images: {len(test_data)}")

# Initialize network
from torchvision.models.segmentation import fcn_resnet50
netAug = fcn_resnet50(pretrained=False, num_classes=1)
netAug.to(device)
optimizer = torch.optim.Adam(netAug.parameters(), lr=learning_rate)

# Training loop
train(netAug, train_data, test_data, criterion, optimizer, batch_size, epochs, "segmentation", lambda out : out["out"])

# TODO calculate segmentation metrics on test set

In [ ]:
# Visualize example image and mask
sample_train_images, sample_train_masks = next(iter(DataLoader(train_data, batch_size=16)))
sample_test_images, sample_test_masks = next(iter(DataLoader(test_data, batch_size=16)))

sample_id = 6
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()

plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()

In [5]:
# Evaluate network on sample images
sample_train_predictions = torch.sigmoid(net(sample_train_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()
sample_test_predictions = torch.sigmoid(net(sample_test_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()

sample_train_predictionsAug = torch.sigmoid(netAug(sample_train_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()
sample_test_predictionsAug = torch.sigmoid(netAug(sample_test_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()

NameError: name 'torch' is not defined

In [1]:
sample_id = 6

# Plot sample from training set
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_predictions[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_predictionsAug[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
print("Without Augmentation:")
print(f"Train sample prediction min: {np.min(sample_train_predictions[sample_id][0])}")
print(f"Train sample prediction max: {np.max(sample_train_predictions[sample_id][0])}")
print("With Augmentation:")
print(f"Train sample prediction min: {np.min(sample_train_predictionsAug[sample_id][0])}")
print(f"Train sample prediction max: {np.max(sample_train_predictionsAug[sample_id][0])}")

# Plot sample from test set
plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_predictions[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_predictionsAug[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
print("Without Augmentation:")
print(f"Test sample prediction min: {np.min(sample_test_predictions[sample_id][0])}")
print(f"Test sample prediction max: {np.max(sample_test_predictions[sample_id][0])}")
print("With Augmentation:")
print(f"Test sample prediction min: {np.min(sample_test_predictions[sample_id][0])}")
print(f"Test sample prediction max: {np.max(sample_test_predictions[sample_id][0])}")

NameError: name 'plt' is not defined

In [3]:
# Evaluate network on sample images
sample_train_predictions = torch.sigmoid(netAug(sample_train_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()
sample_test_predictions = torch.sigmoid(netAug(sample_test_images.expand(-1, 3, -1, -1).to(device))["out"]).detach().cpu().numpy()

NameError: name 'torch' is not defined

In [4]:
sample_id = 6

# Plot sample from training set
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_train_images[sample_id][0])
plt.imshow(sample_train_predictions[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
print(f"Train sample prediction min: {np.min(sample_train_predictions[sample_id][0])}")
print(f"Train sample prediction max: {np.max(sample_train_predictions[sample_id][0])}")

# Plot sample from test set
plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_masks[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
plt.imshow(sample_test_images[sample_id][0])
plt.imshow(sample_test_predictions[sample_id][0], cmap="jet", alpha=0.3)
plt.show()
print(f"Test sample prediction min: {np.min(sample_test_predictions[sample_id][0])}")
print(f"Test sample prediction max: {np.max(sample_test_predictions[sample_id][0])}")

NameError: name 'plt' is not defined